# 12.4 硬件环境
- **目录**
  - 12.4.1 计算机
  - 12.4.2 内存
  - 12.4.3 存储器
    - 12.4.3.1 硬盘驱动器
    - 12.4.3.2 固态驱动器
    - 12.4.3.3 云存储
  - 12.4.4 CPU
    - 12.4.4.1 微体系结构
    - 12.4.4.2 矢量化
    - 12.4.4.3 缓存
  - 12.4.5 GPU和其他加速卡
  - 12.4.6 网络和总线
  - 12.4.7 更多延迟信息

- 很好地理解算法和模型才可以捕获统计方面的问题，构建出具有出色性能的系统。
- 同时，至少对**底层硬件**有一定的了解也是必不可少的。
- 本节不能替代硬件和系统设计的相关课程。
- 相反，本节的内容可以作为理解某些算法为什么比其他算法更高效以及如何实现**良好吞吐量**的起点。
- 一个好的设计可以很容易地在性能上造就**数量级的差异**，这也是后续产生的能够训练网络（例如，训练时间为周）和无法训练网络（训练时间为个月，导致错过截止期）之间的差异。
- 我们先从计算机的研究开始。
- 然后深入查看CPU和GPU。
- 最后，再查看数据中心或云平台上的多台计算机的连接方式。

<center><img src='../img/latencynumbers.png' /></center>
<center>图12.4.1 每个程序员都应该知道的延迟数字</center>

- 你也可以通过 图12.4.1进行简单的了解，图片源自科林·斯科特的[互动帖子](https://people.eecs.berkeley.edu/~rcs/research/interactive_latency.html)，在帖子中很好地概述了过去十年的进展。
- 原始的数字是取自于杰夫迪恩的[Stanford讲座](https://static.googleusercontent.com/media/research.google.com/en//people/jeff/Stanford-DL-Nov-2010.pdf)。
- 下面的讨论解释了这些数字的一些基本原理，以及它们如何指导算法的设计。

## 12.4.1 计算机
- 大多数深度学习研究者和实践者都可以使用一台具有相当数量的内存、计算资源、某种形式的加速器（如一个或者多个GPU）的计算机。计算机由以下关键部件组成：
  * 一个处理器（也被称为CPU），它除了能够运行操作系统和许多其他功能之外，还能够执行我们给它的程序，通常由<b>$8$个或更多个核心组成</b>。
  * 内存（随机访问存储，RAM）用于存储和检索计算结果，如权重向量和激活参数，以及训练数据。
  * 一个或多个以太网连接，速度从1GB/s到100GB/s不等。在高端服务器上可能用到更高级的互连。
  * 高速扩展总线（PCIe, Peripheral Component Interconnect express）用于系统连接一个或多个GPU。服务器**最多有$8$个加速卡**，通常以更高级的拓扑方式连接，而桌面系统则有$1$个或$2$个加速卡，具体取决于用户的预算和电源负载的大小。
  * 持久性存储设备，如磁盘驱动器、固态驱动器，在许多情况下使用高速扩展总线连接。它为系统需要的训练数据和中间检查点需要的存储提供了足够的传输速度。
<center><img src='../img/mobo-symbol.svg'></center>
<center>图12.4.2 计算机组件的连接</center>

- 如图12.4.2所示，高速扩展总线由直接连接到CPU的多个通道组成，将CPU与大多数组件（网络、GPU和存储）连接在一起。
  - 例如，AMD的Threadripper3有$64$个PCIe4.0通道，每个通道都能够双向传输16Gbit/s的数据。
  - 内存直接连接到CPU，总带宽高达100GB/s。
- 当我们在计算机上运行代码时，需要将数据转移到处理器上（CPU或GPU）执行计算，然后将结果从处理器移回到随机访问存储和持久存储器中。
- 因此，为了获得良好的性能，需要确保每一步工作都能无缝链接，而不希望系统中的任何一部分成为主要的瓶颈。
  - 例如，如果不能快速加载图像，那么处理器就无事可做。
- 同样地，如果不能快速移动矩阵到CPU（或GPU）上，那么CPU（或GPU）就会无法全速运行。
- 最后，如果希望在网络上**同步多台计算机**，那么网络就不应该拖累计算速度。
  - 一种选择是通信和计算交错进行。
- 接下来将详细地介绍各个组件。

- **要点：**
  - **处理器（CPU）**：执行程序和运行操作系统的关键部件，通常由8个或更多个核心组成。
  - **内存（RAM）**：用于存储和检索计算结果，如权重向量、激活参数和训练数据。
  - **以太网连接**：允许不同速度的数据通信，从1GB/s到100GB/s不等。
  - **高速扩展总线（PCIe）**：用于连接一个或多个GPU。服务器可能有多达8个加速卡，而桌面系统可能有1到2个加速卡。
  - **持久性存储设备**：例如硬盘驱动器和固态驱动器，为训练数据和中间检查点提供足够的传输速度。
  - **高速扩展总线的结构**：连接CPU与大部分组件，例如AMD的Threadripper3拥有PCIe4.0通道，可以双向传输16Gbit/s的数据。
  - **内存和CPU的连接**：直接连接，总带宽高达100GB/s。
  - **数据转移的重要性**：在运行代码时，需要将数据转移到CPU或GPU上执行计算，并将结果移回到RAM和持久存储器中。
  - **避免系统瓶颈**：为了优化性能，每个部分都必须无缝链接，无论是加载图像的速度、移动矩阵到CPU或GPU的速度，还是同步多台计算机的网络速度。
  - **通信和计算的交错**：一种提高效率的选择，确保通信不会成为计算速度的限制。

## 12.4.2 内存

- 最基本的内存主要用于存储需要随时访问的数据。
  - 目前，CPU的内存通常为DDR4类型(DDR:Double Data Rate Memory)，每个模块提供20-25Gb/s的带宽。
    - 视频制作时已发展到DDR5，时钟频率为1600-3200 MHz，带宽为25.6-51.2 GB/s。
  - 每个模块都有一条$64$位宽的总线。通常使用成对的内存模块来允许多个通道。
  - CPU有$2$到$4$个内存通道，也就是说，它们内存带宽的峰值在40GB/s到100GB/s之间。
  - 一般每个通道有两个物理存储体（bank）。例如AMD的Zen 3 Threadripper有$8$个插槽。
- 当我们想要从内存中读取一部分内容时，我们需要先告诉内存模块在哪里可以找到信息。
- 也就是说，我们需要先将**地址（address）** 发送到RAM。然后我们可以选择只读取一条$64$位记录还是一长串记录。
- 后者称为**突发读取（burst read）**。
  - 概括地说，向内存发送地址并设置传输大约需要100ns（细节取决于所用内存芯片的特定定时系数），每个后续传输只需要0.2ns。
- 总之，**第一次读取的成本是后续读取的500倍**！
- 请注意，我们每秒最多可以执行一千万次随机读取。
- 这说明应该尽可能地避免随机内存访问，而是使用突发模式读取和写入。

- 当考虑多个物理存储体时，事情就更加复杂了。
- 每个存储体大部分时候都可以独立地读取内存。这意味着两件事。
  - 一方面，如果随机读操作均匀分布在内存中，那么有效的随机读操作次数将高达4倍。这也意味着执行随机读取仍然不是一个好主意，因为突发读取的速度也快了4倍。
  - 另一方面，由于内存对齐是$64$位边界，因此最好将任何数据结构与相同的边界对齐。
  - 当设置了适当的标志时，编译器基本上就是自动化地执行对齐操作。
- 因为GPU的处理单元比CPU多得多，因此它对内存带宽的需要也更高。
- 解决这种问题大体上有两种选择。
  - 首要方法是使内存总线变得更宽。例如：NVIDIA的RTX 2080Ti有一条$352$位宽的总线（Memory Bus Width，**A100的总线位宽为512-bit**），这样就可以同时传输更多的信息。再有方法就是在GPU中使用特定的高性能内存。
  - 一种选择是如NVIDIA的消费级设备RTX和Titan系列中通常使用GDDR6芯片，其总带宽超过500GB/s。
  - 另一种选择是使用HBM（高带宽存储器)模块。这些模块使用截然不同的接口在专用硅片上与GPU直接连在一起。这导致其非常昂贵，通常仅限于在高端服务器的芯片上使用，如NVIDIA Volta V100系列的加速卡。

- GPU内存的带宽要求甚至更高，因为它们的处理单元比CPU多得多。
  - 总的来说，解决这些问题有两种选择。首先是使内存总线变得更宽。
    - 例如，NVIDIA的RTX 2080Ti有一条352位宽的总线。这样就可以同时传输更多的信息。
  - 其次，GPU使用特定的高性能内存。消费级设备，如NVIDIA的RTX和Titan系列，通常使用GDDR6芯片，总带宽超过500GB/s。
  - 另一种选择是使用HBM（高带宽存储器)模块。它们使用截然不同的接口，直接与专用硅片上的GPU连接。
  - 这使得它们非常昂贵，通常仅限于高端服务器芯片，如NVIDIA Volta V100系列加速卡。
- 毫不意外的是GPU的内存通常比CPU的内存小得多，因为前者的成本更高。
- 就目的而言，它们的性能与特征大体上是相似的，只是GPU的速度更快。
- 就本书而言，我们完全可以忽略细节，因为这些技术只在调整GPU核心以获得高吞吐量时才起作用。

- **要点：**
  - **内存类型和带宽**：
    - CPU通常使用DDR4内存，每个模块提供20-25Gb/s带宽。
    - 有2到4个内存通道，每个64位宽，峰值带宽在40GB/s到100GB/s之间。
    - 例如，AMD的Zen 3 Threadripper有8个插槽。
  - **读取过程和突发读取**：
    - 读取数据前需要先发送地址到RAM。
    - 可选择读取一条64位记录或突发读取一长串记录，后者称为突发读取。
    - 第一次读取与后续读取的成本差异巨大（约500倍）。
  - **随机读取与突发模式**：
    - 每秒最多可执行一千万次随机读取，但应尽量避免随机访问，使用突发模式读取和写入。
  - **物理存储体的复杂性**：
     - 每个存储体大部分时间可以独立读取内存。
     - 随机读操作的有效次数可能增加，但突发读取的速度也快了。
     - 最佳做法是将数据结构与64位边界对齐。
  - **GPU对内存需求**：
    - GPU的处理单元比CPU多，对内存带宽需求更高。
    - 解决方案包括更宽的内存总线，例如352位宽的总线，或使用特殊高性能内存（例如GDDR6或HBM）。
  - **特殊的GPU内存选项**：
    - 如NVIDIA的消费级设备通常使用GDDR6芯片，带宽超过500GB/s。
    - 或使用高带宽存储器（HBM）模块，通常限于高端服务器芯片。
  - **GPU与CPU内存的比较**：
    - GPU的内存通常比CPU的小，但速度更快。
    - 在调整GPU核心以获得高吞吐量时，可能需要考虑细微差别，但在大多数情况下，可以忽略这些细节。

## 12.4.3 存储器

我们看到随机访问存储的一些关键特性是 **带宽（bandwidth）**和**延迟（latency）**。存储设备也是如此，只是不同设备之间的特性差异可能更大。

### 12.4.3.1 硬盘驱动器

**硬盘驱动器**（hard disk drive，HDD）已经使用了半个多世纪。简单的说，它们包含许多旋转的盘片，这些盘片的磁头可以放置在任何给定的磁道上进行读写。高端磁盘在$9$个盘片上可容纳高达16TB的容量。硬盘的主要优点之一是相对便宜，而它们的众多缺点之一是典型的灾难性故障模式和相对较高的读取延迟。

要理解后者，请了解一个事实即硬盘驱动器的转速大约为7200RPM（每分钟转数）。它们如果转速再快些，就会由于施加在碟片上的离心力而破碎。在访问磁盘上的特定扇区时，还有一个关键问题：需要等待碟片旋转到位（可以移动磁头，但是无法对磁盘加速）。因此，可能需要$8$毫秒才能使用请求的数据。一种常见的描述方式是，硬盘驱动器可以以大约100IOPs（每秒输入/输出操作）的速度工作，并且在过去二十年中这个数字基本上没变。同样糟糕的是，带宽（大约为100-200MB/s）也很难增加。毕竟，每个磁头读取一个磁道的比特，因此比特率只随信息密度的平方根缩放。因此，对于非常大的数据集，HDD正迅速降级为归档存储和低级存储。

### 12.4.3.2 固态驱动器

**固态驱动器**（solid state drives，SSD）使用闪存持久地存储信息。这允许更快地访问存储的记录。现代的固态驱动器的IOPs可以达到$10$万到$50$万，比硬盘驱动器快3个数量级。而且，它们的带宽可以达到1-3GB/s，比硬盘驱动器快一个数量级。这些改进听起来好的难以置信，而事实上受固态驱动器的设计方式，它仍然存在下面的附加条件：

* 固态驱动器以块的方式（256KB或更大）存储信息。块只能作为一个整体来写入，因此需要耗费大量的时间，导致固态驱动器在按位随机写入时性能非常差。而且通常数据写入需要大量的时间还因为块必须被读取、擦除，然后再重新写入新的信息。如今固态驱动器的控制器和固件已经开发出了缓解这种情况的算法。尽管有了算法，写入速度仍然会比读取慢得多，特别是对于QLC（四层单元）固态驱动器。提高性能的关键是维护操作的“队列”，在队列中尽可能地优先读取和写入大的块。
* 固态驱动器中的存储单元磨损得比较快（通常在几千次写入之后就已经老化了）。磨损程度保护算法能够将退化平摊到许多单元。也就是说，不建议将固态驱动器用于交换分区文件或大型日志文件。
* 最后，带宽的大幅增加迫使计算机设计者将固态驱动器与PCIe总线相连接，这种驱动器称为NVMe（非易失性内存增强），其最多可以使用$4$个PCIe通道。在PCIe4.0上最高可达8GB/s。

### 12.4.3.3 云存储

云存储提供了一系列可配置的性能。也就是说，虚拟机的存储在数量和速度上都能根据用户需要进行动态分配。我们建议用户在延迟太高时（例如，在训练期间存在许多小记录时）增加IOPs的配置数。

- **要点：**
  - **硬盘驱动器 (HDD)**
    - 已使用超过半个世纪，包含旋转盘片，使用磁头读写。
    - 高端磁盘可容纳16TB的容量。
    - 主要优点：相对便宜。
    - 主要缺点：灾难性故障模式和较高读取延迟。
    - 转速约7200RPM，访问特定扇区可能需要$8$毫秒。
    - 每秒输入/输出操作 (IOPs) 约为100，带宽约为100-200MB/s。
    - 对于非常大的数据集，逐渐变为归档存储和低级存储。
  - **固态驱动器 (SSD)**
    - 使用闪存持久存储信息，允许更快访问。
    - IOPs可达$10$万到$50$万，比HDD快3个数量级。
    - 带宽可以达到1-3GB/s，比HDD快一个数量级。
    - 以块方式存储信息，导致按位随机写入性能差。
    - 存储单元磨损较快，有磨损程度保护算法。
    - NVMe通过PCIe总线连接，带宽最高可达8GB/s。
  - **云存储**
    - 提供可配置性能，数量和速度可根据需要动态分配。
    - 在延迟过高时，建议增加IOPs配置数。

## 12.4.4 CPU

- **中央处理器（central processing unit，CPU）** 是任何计算机的核心。
- 它们由许多关键组件组成：
  - **处理器核心（processor cores）** 用于执行机器代码的、
  - **总线（bus）** 用于连接不同组件（注意，总线会因为处理器型号、各代产品和供应商之间的特定拓扑结构有明显不同）
  - **缓存（cach）** 相比主内存实现更高的读取带宽和更低的延迟内存访问。
  - 最后，因为高性能线性代数和卷积运算常见于媒体处理和机器学习中，所以几乎所有的现代CPU都包含**向量处理单元（vector processing unit）** 为这些计算提供辅助。
<center><img src='../img/skylake.svg'></center>
<center>图12.4.3 Intel Skylake消费级四核CPU</center>
- 图12.4.3描述了Intel **Skylake消费级四核CPU** ：
  - 它包含一个集成GPU、缓存和一个连接四个核心的环总线。
  - 例如：以太网、WiFi、蓝牙、SSD控制器和USB这些外围设备要么是芯片组的一部分，要么通过PCIe直接连接到CPU。

----
- **说明：Intel Skylake CPU结构**
  - **1. Skylake CPU核心部分（紫色区域）**
    - **四个Skylake CPU核心：** 每个核心都有独立的L3缓存，负责处理指令和数据运算。
    - **L3缓存：** 每个核心都有独立的L3缓存，用于存储核心数据并加速访问。多个核心通过高速互联总线连接，共享数据。
  - **2. 显示单元（绿色和蓝色区域）**
    - **HD 500 Graphics Unit（绿色）：** 集成的图形处理单元（GPU），支持图形渲染和视频加速任务。
    - **Display和视频模块：** 负责处理显示输出，支持最高4K显示分辨率。
  - **3. 内存控制器（MC，蓝色区域）**
    - **DIMMs接口：** 支持双通道DDR3/DDR4内存，显著提高内存带宽。
    - **eDRAM：** 集成了64MB或128MB的嵌入式DRAM，作为高速缓存存储器，用于提高内存访问速度。
  - **4. 北桥（North Bridge，蓝色区域）**
    - **DMI/OPI接口：**  连接南桥芯片，负责数据传输。
    - **PCIe 3.0接口：** 提供高速通道给外部设备，比如独立显卡。
  - **5. 南桥（South Bridge，绿色区域）**
    - **Intel 100系列芯片组：**  管理I/O设备和外部接口，包括Wi-Fi、蓝牙、以太网和Thunderbolt等。
  - **6. 外部接口（粉红色区域）**
    - **Wi-Fi + Bluetooth：** 无线网络和蓝牙连接模块。
    - **Ethernet（以太网）：**  提供有线网络连接。
    - **Thunderbolt：** 高速数据传输和视频输出接口。
    - **WiGig：**  无线千兆数据传输技术。
  - **7. 其他模块**
    - **ISP（图像信号处理器）：** 处理图像相关任务，比如摄像头图像的增强和输出。
    - **视频模块：** 专门用于视频解码和编码。
----

### 12.4.4.1 微体系结构

每个处理器核心都由一组相当复杂的组件组成。虽然不同时代的产品和供应商的细节有所不同，但基本功能都是标准的。前端加载指令并尝试预测将采用哪条路径（例如，为了控制流），然后将指令从汇编代码解码为微指令。汇编代码通常不是处理器执行的最低级别代码，而复杂的微指令却可以被解码成一组更低级的操作，然后由实际的执行核心处理。通常执行核心能够同时执行许多操作，例如， 图12.4.4的ARM Cortex A77核心可以同时执行多达个操作。ARM全称**Advanced RISC Machines**，即**高级精简指令集计算机**。看看资本和技术的精彩博弈过程，挺有意思的，[视频网址](https://www.bilibili.com/video/BV1bumyYaEyQ/?spm_id_from=333.337.search-card.all.click&vd_source=f229b48b5ab83786d3217870304efe4d)。

<center><img src='../img/a77.svg'></center>
<center>图12.4.4 ARM Cortex A77微体系结构</center>
<br>
这意味着高效的程序可以<b>在每个时钟周期内执行多条指令</b> ，前提是这些指令可以独立执行。不是所有的处理单元都是平等的。一些专用于处理整数指令，而另一些则针对浮点性能进行了优化。为了提高吞吐量，处理器还可以在分支指令中同时执行多条代码路径，然后丢弃未选择分支的结果。这就是为什么前端的分支预测单元很重要，因为只有最有希望的路径才会被继续执行。

- **要点：**
  - **核心组成**：每个处理器核心由一组复杂的组件组成，虽然不同时代和供应商的细节有所不同，但基本功能是标准的。
  - **前端工作**：前端负责加载指令，并尝试预测将要采用的路径（如控制流）。其将指令从汇编代码解码成微指令。
  - **微指令和执行**：汇编代码通常不是处理器执行的最低级别代码。复杂的微指令可以被解码成更低级的操作，由执行核心处理。执行核心通常能同时执行多个操作。
  - **ARM Cortex A77实例**：例如，ARM Cortex A77核心的微体系结构可以同时执行多达个操作。
  - **指令并行执行**：高效的程序可以在每个时钟周期内执行多条指令，前提是这些指令可以独立执行。
  - **不同的处理单元**：不是所有的处理单元都是平等的。一些专门处理整数指令，另一些则针对浮点性能优化。
  - **提高吞吐量的策略**：为了提高吞吐量，处理器可以在分支指令中同时执行多条代码路径，然后丢弃未选择分支的结果。
  - **分支预测单元的重要性**：前端的分支预测单元很重要，因为只有最有希望的路径才会被继续执行。

---
- **说明：ARM Cortex A77微体系结构**
  - **1. 主要流程**
    - （1） **取指（Fetch）**：处理器从指令缓存或内存中取出指令。
    - （2） **译码、重命名、派遣（Decode, Rename, Dispatch）**
      - **译码（Decode）：** 解析指令，确定其含义。
      - **重命名（Rename）：** 处理寄存器重命名，以避免数据相关性问题。
      - **派遣（Dispatch）：** 将指令分配到合适的执行单元。
    - （3）**发射（Issue）**
      - 指令进入发射队列，等待分配到合适的执行单元。
    - （4）**执行单元（Execution Units）**
      - 指令会被调度到不同的执行单元完成计算任务。
  - **2. 执行单元分类**执行单元负责处理不同类型的指令，主要包括如下几类：
    - （1） **分支执行单元（Branch Unit）**
      - 处理分支指令，如跳转、条件执行等。
      - 该图中含有 **“分支0”** 和 **“分支1”** 两个分支执行单元。
    - （2）**整数执行单元（Integer ALU）** 处理整数运算，如加减乘除、逻辑运算等。该架构包含：
      - **整数单周期0**（Integer Single Cycle 0）
      - **整数单周期1**（Integer Single Cycle 1）
      - **整数单/多周期0**（Integer Single/Multi Cycle 0）
      - **整数单/多周期1**（Integer Single/Multi Cycle 1）
    - （3）**浮点和 SIMD 执行单元（FP/ASIMD Units）** 处理 **浮点运算（FP, Floating Point）** 和 **高级 SIMD（ASIMD, Advanced SIMD）** 指令。该架构包含：
      - **FP/ASIMD 0**
      - **FP/ASIMD 1**
    - （4）**加载/存储单元（Load/Store Units）** 负责数据的加载和存储操作。该架构包含：
      - **加载/存入0（Load/Store 0）**
      - **加载/存入1（Load/Store 1）**
      - **存数据0（Store Data 0）**
      - **存数据1（Store Data 1）**
  - **3. 结构分析**
    - **宽度**：Cortex A77 采用 **8 发射（Dispatch）** 设计，意味着每个周期最多可以向不同的执行单元发射 8 条指令，提高并行度。
    - **执行并行度**：
      - 2 个分支执行单元
      - 4 个整数执行单元
      - 2 个 FP/ASIMD 单元
      - 2 个加载/存储单元
      - 2 个存数据单元
    - **高效流水线**：
      - Cortex A77 支持 **乱序执行（Out-of-Order Execution）**，提高指令吞吐量。
      - **寄存器重命名** 解决数据相关性问题，减少流水线停滞。
---

### 12.4.4.2 矢量化

- 深度学习的计算量非常大，为了满足机器学习的需要，CPU需要在**一个时钟周期内执行许多操作**。
- 这种执行方式是通过**向量处理单元**实现的。
  - 这些处理单元有不同的名称:在ARM上叫做NEON,在x86上被称为AVX2。
  - 一个常见的功能是它们能够执行**单指令多数据（single instruction multiple data，SIMD）** 操作。
- 图12.4.5显示了如何在ARM上的一个时钟周期中完成$8$个整数加法。
<center><img src='../img/neon128.svg' /></center>
<center>图12.4.5 128位NEON矢量化</center>
<br>
- 根据体系结构的选择，此类寄存器最长可达$512$位，最多可组合$64$对数字。
  - 例如，我们可能会将两个数字相乘，然后与第三个数字相加，这也称为**乘加融合（fused multiply-add）**。
- Intel的OpenVino就是使用这些处理器来获得可观的吞吐量，以便在服务器级CPU上进行深度学习。
- 不过请注意，这个数字与GPU的能力相比则相形见绌。
  - 例如，NVIDIA的A100拥有$8192$个CUDA核心，每个核心都能够在任何时候处理这样的操作。

--------
- **说明：128位NEON矢量化**
  - **1. 图的组成部分**
    - **Q2、Q1、Q0**：
      - 图中的 Q2、Q1、Q0 是 NEON 寄存器中的 128 位矢量寄存器。
      - 每个寄存器可以存储多个数据元素（比如 8 位、16 位或 32 位整数）。
      - 图中 Q2、Q1、Q0 被分成了 8 个子块，每个子块表示 16 位（2 个字节）的数据。
    - **蓝色区域（Q1）**：
      - 蓝色部分表示当前执行矢量运算的输入数据来源寄存器 Q1。
      - 该寄存器中的每个 16 位数据将与其他寄存器（如 Q2）中的对应数据进行矢量化操作。
    - **加号圆圈（+）**：
      - 表示加法操作（矢量加法）。
      - 每个圆圈对应于一对数据元素的加法操作。
    - **箭头**：
      - 表示数据流，即输入数据从 Q2 和 Q1 提取，经过加法操作后，结果存入目标寄存器 Q0 中。
  - **2. 运算过程**
    - **输入数据**：
      - 两个源寄存器：Q2 和 Q1，每个寄存器包含 8 个 16 位整数。
      - 例如：
        - Q2 = [a7, a6, a5, a4, a3, a2, a1, a0]
        - Q1 = [b7, b6, b5, b4, b3, b2, b1, b0]
        - 每个元素的下标表示其在寄存器中的位置。
    - **操作**：
      - 每对对应位置的数据元素（如 Q2 的 a0 和 Q1 的 b0）进行加法运算。
      - 运算结果存储在目标寄存器 Q0 的对应位置上。
      - 例如：
        - Q0[0] = Q2[0] + Q1[0] （a0 + b0）
        - Q0[1] = Q2[1] + Q1[1] （a1 + b1）
        - ...
    - **结果数据**：
      - 运算完成后，目标寄存器 Q0 中存储了对应位置的加法结果：
        - Q0 = [a7+b7, a6+b6, a5+b5, a4+b4, a3+b3, a2+b2, a1+b1, a0+b0]
  - **3. 关键概念**
    - **矢量化运算**：
      - NEON 的矢量化运算可以一次性对多个数据元素进行并行计算，提高数据处理效率。
      - 图中展示的是一个 128 位寄存器的情况，支持 8 个 16 位数据的并行计算。
    - **并行性**：
      - 每个加法运算（+）是独立的，可以并行完成。
      - 这种并行处理能力是 NEON 指令集实现高性能计算的核心优势。
    - **128 位宽度**：
      - 图中每个寄存器宽度为 128 位，表示可以容纳 8 个 16 位整数（或其他类型的数据，如 16 个 8 位整数）。
  - **4. 应用场景**
    - **多媒体处理**：
      - 图像和视频处理中的像素加法（如亮度调整）。
    - **数字信号处理**：
      - 处理音频波形或滤波器运算。
    - **机器学习**：
      - 神经网络中的矢量化加权和运算。
-------

###  12.4.4.3 缓存

考虑以下情况：我们有一个中等规模的核心的CPU，如 图12.4.3所示，运行在2GHz频率。此外，假设向量处理单元启用了256位带宽的AVX2(Advanced Vector Extensions 2)，其IPC（指令/时钟）计数为1。进一步假设从内存中获取用于AVX2操作的指令至少需要一个寄存器。这意味着CPU每个时钟周期需要消耗$4 \times 256  bits = 128 bytes$的数据。除非我们能够每秒向处理器传输$2 \times 10^9 \times 128 = 256 \times 10^9$字节，否则用于处理的数据将会不足。不幸的是，这种芯片的存储器接口仅支持20-40Gb/s的数据传输，即少了一个数量级。解决方法是尽可能避免从内存中加载新数据，而是将数据放在CPU的缓存上。这就是使用缓存的地方。

通常使用以下名称或概念:

- 寄存器，严格来说不是缓存的一部分，用于帮助**组织指令**。也就是说，**寄存器是CPU可以以时钟速度访问而没有延迟的存储位置**。CPU有几十个寄存器，因此有效地使用寄存器取决于编译器（或程序员）。例如，C语言有一个register关键字。
- 一级缓存是应对高内存带宽要求的第一道防线。一级缓存很小（常见的大小可能是32-64KB），内容通常分为数据和指令。当数据在一级缓存中被找到时，其访问速度非常快，如果没有在那里找到，搜索将沿着缓存层次结构向下寻找。

- 二级缓存是下一站。根据架构设计和处理器大小的不同，它们可能是独占的也可能是共享的。即它们可能只能由给定的核心访问，或者在多个核心之间共享。二级缓存比一级缓存大（通常每个核心256-512KB），而速度也更慢。此外，我们首先需要检查以确定数据不在一级缓存中，才会访问二级缓存中的内容，这会增加少量的额外延迟。

- 三级缓存在多个核之间共享，并且可以非常大。AMD的EPYC 3服务器的CPU在多个芯片上拥有高达256MB的高速缓存。更常见的数字在4-8MB范围内。

预测下一步需要哪个存储设备是优化芯片设计的关键参数之一。例如，建议以**向前的方向遍历内存**，因为大多数缓存算法将试图向前读取（read forward）而不是向后读取。同样，将内存访问模式保持在本地也是提高性能的一个好方法。

添加缓存是一把双刃剑。一方面，它能确保处理器核心不缺乏数据。但同时，它也增加了芯片尺寸，消耗了原本可以用来提高处理能力的面积。

此外，缓存未命中的代价可能会很昂贵。考虑最坏的情况，如 图12.4.6所示的错误共享（false sharing）。当处理器上的线程请求数据时，内存位置缓存在处理器上。为了满足获取需要，**处理器需要停止它正在做的事情，将信息写回主内存，然后让处理器从内存中读取它。在此操作期间，两个处理器都需要等待**。与高效的单处理器实现相比，这种代码在多个处理器上运行的速度可能要慢得多。这就是为什么缓存大小（除了物理大小之外）有实际限制的另一个原因。

<center><img src='../img/falsesharing.svg' /></center>
<center>图12.4.6 错误共享（图片由英特尔提供）</center>
<br>

- **要点：**
  - **缓存的需求**：当CPU在每个时钟周期中需要消耗大量数据，且内存带宽无法满足时，缓存成为了必需。由于芯片的存储器接口可能仅支持较低的数据传输速率，因此需要将数据存储在CPU的缓存中以避免瓶颈。
  - **缓存层级**：
     - **寄存器**：虽然不是缓存的一部分，但寄存器为CPU提供了时钟速度访问的存储位置，没有延迟。
     - **一级缓存**：小型缓存（通常为32-64KB），用作应对高内存带宽要求的第一道防线。内容通常分为数据和指令，访问速度非常快。
     - **二级缓存**：较大（通常每个核心256-512KB），可能是独占或共享的，速度更慢，访问前需要检查一级缓存。
     - **三级缓存**：在多个核之间共享，可以非常大，例如256MB或更常见的4-8MB范围。
  - **存储设备预测**：预测下一步需要哪个存储设备是优化芯片设计的关键。通常，向前遍历内存和保持内存访问模式在本地是提高性能的有效方法。
  - **缓存的双重性质**：
     - **优势**：缓存可以确保处理器核心不缺乏数据，提高效率。
     - **劣势**：增加了芯片尺寸，消耗了提高处理能力的空间，并且缓存未命中的代价可能很昂贵。
  - **错误共享问题**：在某些情况下，错误共享（false sharing）可能导致多个处理器等待，从而降低多处理器执行代码的效率。这解释了为什么缓存大小会有实际限制。
  - **编译器和编程的角色**：有效使用寄存器以及编程方式可以影响缓存效率。例如，C语言中的`register`关键字可以影响寄存器的使用。

----
- **说明：错误共享(False Sharing)详解**
  - 元素解释：
    - **Socket 0 和 Socket 1**：表示两个不同的处理器插槽，每个插槽有一个线程（Thread 0 和 Thread 1）。
    - **Cache**：每个插槽都有自己的缓存，用于存储和快速访问数据。
    - **Cache Line**：缓存中的基本单位，通常是64字节。图中显示了两个缓存行，每个缓存行包含多个小格子代表数据。
  - 错误共享现象：
    - **共享缓存行**：两个线程（Thread 0 和 Thread 1）正在访问同一缓存行中的不同数据。
    - **线程冲突**：尽管线程访问的是不同的数据，由于它们位于同一缓存行内，导致两个插槽的缓存频繁失效（Cache Miss）。
    - **数据传输**：当一个线程修改缓存行中的数据时，整个缓存行需要在不同的缓存之间传输，导致性能下降。
  - 数据流分析：
    - **红色箭头**：表示数据从内存载入到缓存的过程。Thread 0 修改了缓存行中的数据，导致该缓存行的更新需要传递到内存。
    - **蓝色箭头**：表示数据从缓存返回到内存或从内存载入到缓存。Thread 1 在访问时，需要从内存中获取更新后的缓存行。
  - 影响：
    - 由于两个线程在不同的插槽上同时访问共享缓存行中的不同数据，频繁的缓存同步操作降低了系统性能。
    - 这种不必要的缓存同步操作就是“错误共享”，它降低了多线程程序的效率。
  - 解决方案：
    - 通过调整数据结构，确保不同线程访问的数据不在同一缓存行内，可以有效减少错误共享现象。

-----

## 12.4.5 GPU和其他加速卡

毫不夸张地说，如果没有GPU，深度学习就不会成功。基于同样的原因，有理由认为GPU制造商的财富由于深度学习而显著增加。这种硬件和算法的**协同进化**导致了这样一种情况：无论好坏，深度学习都是更可取的**统计建模范式**。因此，了解GPU和其他加速卡（如TPU）的具体好处是值得的。

值得注意的是，在实践中经常会有这样一个判别：加速卡是为**训练**还是**推断**而优化的。对于**后者（即推断），我们只需要计算网络中的前向传播**。而反向传播不需要存储中间数据。还有，我们可能不需要非常精确的计算（FP16或INT8通常就足够了）。对于前者，即**训练过程中需要存储所有的中间结果用来计算梯度**。而且，累积梯度也需要更高的精度，以避免数值下溢（或溢出）。这意味着最低要求也是FP16（或FP16与FP32的混合精度）。所有这些都需要更快、更大的内存（HBM2或者GDDR6）和更高的处理能力。例如，NVIDIA优化了[Turing](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/) T4 GPU用于推断和V100 GPU用于训练。

回想一下如 图12.4.5所示的矢量化。处理器核心中添加向量处理单元可以显著提高吞吐量。例如，在 图12.4.5的例子中，我们能够同时执行16个操作。首先，如果我们添加的运算不仅**优化了向量运算，而且优化了矩阵运算**，会有什么好处？稍后我们将讨论基于这个策略引入的张量核（tensor cores）。第二，如果我们增加更多的核心呢？简而言之，以上就是GPU设计决策中的两种策略。 图12.4.7给出了基本处理块的概述。它包含**16个整数单位和16个浮点单位**。除此之外，两个张量核加速了与深度学习相关的附加操作的狭窄的子集。每个流式多处理器都由这样的四个块组成。

<center><img src='../img/turing-processing-block.png' height=300px width=300px /></center>
<center>图12.4.7 NVIDIA Turing处理块（图片由英伟达提供））</center>
<br>

接下来，将12个流式多处理器分组为图形处理集群，这些集群构成了高端TU102处理器。充足的内存通道和二级缓存完善了配置。 图12.4.8有相关的细节。设计这种设备的原因之一是可以根据需要独立地添加或删除模块，从而满足设计更紧凑的芯片和处理良品率问题（故障模块可能无法激活）的需要。幸运的是，在CUDA和框架代码层之下，这类设备的编程对深度学习的临时研究员隐藏得很好。特别是，只要有可用的资源GPU上就可以同时执行多个程序。尽管如此，了解设备的局限性是值得的，以避免对应的设备内存的型号不合适。

<center><img src='../img/turing.png' height=300px width=300px/></center>
<center>图12.4.8 NVIDIA Turing架构（图片由英伟达提供））</center>
<br>

----
- **说明：NVIDIA Turing架构解释**
  - 图12.4.8展示的是NVIDIA Turing架构的细节。这种架构用于NVIDIA的图形处理单元（GPU），以增强图形计算和并行计算性能。
  - 1. PCI Express 3.0 Host Interface
    - 图的顶部是PCI Express 3.0主机接口，它负责GPU与主机系统（如CPU和内存）之间的通信。
  - 2. GigaThread Engine
    - 紧接在PCI Express接口下方的是GigaThread Engine，它负责管理和调度GPU内的线程，确保高效的并行计算。
  - 3. GPC（Graphics Processing Cluster），
    - 图中有多个GPC模块，每个GPC包含多个TPC（Texture Processing Cluster）。GPC是图形处理的主要单位，每个GPC可以独立处理图形任务。
    - 每个GPC由多个TPC（Texture Processing Cluster）组成，每个TPC包含两个SM（Streaming Multiprocessor）。
  - 4. SM（Streaming Multiprocessor）
    - SM是执行单元的核心，每个SM包含多个CUDA内核、Tensor内核和RT（光线追踪）内核。它们负责执行并行计算任务，如图形渲染和科学计算。
    - 4.1 CUDA Cores执行浮点和整数运算的核心单元。它们负责处理大部分计算任务。
    - 4.2 Tensor Cores专门用于深度学习任务，加速矩阵运算。
    - 4.3 RT Cores用于实时光线追踪，显著提高光线追踪的性能。
  - 5. Memory Controllers，在GPC模块的左右两侧是内存控制器，它们管理GPU与显存之间的数据传输。
  - 6. L2 Cache，在GPC模块下方是L2缓存，它是一个共享缓存，用于提高数据访问速度，减少延迟。
  - 7. High-Speed Hub，L2缓存下方是高速枢纽，负责协调不同模块之间的数据传输。
  - 8. NVLink，图的底部是NVLink接口，它用于GPU之间的高速互联，增强多GPU系统的性能。
  - 总结
    - NVIDIA Turing架构通过多个GPC、TPC和SM模块，以及专用的CUDA内核、Tensor内核和RT内核，实现高效的图形处理和并行计算。
    - 这种架构设计使得Turing GPU在图形渲染、科学计算和深度学习等领域具有强大的性能表现。
-----

最后值得一提的是**张量核（tensor core）**。它们是最近增加更多优化电路趋势的一个例子，这些优化电路对深度学习特别有效。例如，TPU添加了用于**快速矩阵乘法**的**脉动阵列**，这种设计是为了支持非常小数量（第一代TPU支持数量为1）的大型操作。而张量核是另一个极端。它们针对和矩阵之间的小型运算进行了优化，具体取决于它们的数值精度。 图12.4.9给出了优化的概述。

<center><img src='../img/tensorcore.jpg' height=400px width=400px/></center>
<center>图12.4.9 NVIDIA Turing架构中的张量核心（图片由英伟达提供）</center>
<br>

显然，我们最终会在优化计算时做出某些妥协。其中之一是**GPU不太擅长处理稀疏数据和中断**。尽管有一些明显的例外，但GPU擅长的高带宽突发读取操作并不适合稀疏的$4 \times 4$矩阵和$16 \times 16$向量的访问模式。访问稀疏数据和处理中断这两个目标是一个积极研究的领域。例如：DGL，一个专为图深度学习而设计的库。

- **要点：**
  - **深度学习与GPU的关联性**：GPU在深度学习的成功中起到了关键作用，也显著增加了GPU制造商的财富（2023年8月15英伟达市值1.08万亿美元，2025年2月20日3.43万亿美元市值）。
    - 硬件和算法的协同进化促成了深度学习成为更可取的统计建模范式。
  - **训练与推断的区别**：在选择和优化加速卡时，一个重要的考虑因素是它们是为训练还是推断而优化的。
    - 训练需要存储所有中间结果以计算梯度，可能需要更高精度；而推断只需前向传播，精度要求可能较低。
  - **加速卡的示例**：NVIDIA的Turing T4 GPU优化用于推断，而V100 GPU优化用于训练。
  - **向量与矩阵运算的优化**：在处理器核心中添加针对向量和矩阵运算的优化可以显著提高吞吐量。
    - 张量核就是基于这种策略的一个实例。
  - **GPU设计的多核心策略**：增加更多的核心是GPU设计决策中的一种策略。
    - 例如，NVIDIA的Turing处理块包含多个整数和浮点单位以及张量核。
  - **模块化构建**：Turing架构中的流式多处理器被分组构成图形处理集群，可独立添加或删除模块，以满足设计和良品率需求。
  - **张量核的特点**：张量核是优化了针对小型矩阵运算的电路，它们对深度学习特别有效。
  - **设备的局限性和挑战**：GPU在处理稀疏数据和中断方面存在局限性。
    - 它们擅长高带宽突发读取操作，但不适合稀疏矩阵和向量的访问模式。处理这些问题是积极研究的领域。
  - **针对特殊需求的库和优化**：有专门的库和设计来满足特定需求，如DGL库专为图深度学习设计。

## 12.4.6 网络和总线

每当单个设备不足以进行优化时，我们就需要来回传输数据以实现同步处理，于是网络和总线就派上了用场。我们有许多设计参数：带宽、成本、距离和灵活性。应用的末端我们有WiFi，它有非常好的使用范围，非常容易使用（毕竟没有线缆），而且还便宜，但它提供的带宽和延迟相对一般。头脑正常的机器学习研究人员都不会用它来构建服务器集群。在接下来的内容中，我们将重点关注适合深度学习的互连方式。

* **PCIe**，一种专用总线，用于每个通道点到点连接的高带宽需求（在$16$通道插槽中的PCIe4.0上高达32GB/s），延迟时间为个位数的微秒（5μs）。PCIe链接非常宝贵。处理器拥有的数量：AMD的EPYC 3有$128$个通道，Intel的Xeon每个芯片有$48$个通道；在桌面级CPU上，数字分别是$20$（Ryzen9）和$16$（Core i9）。由于GPU通常有$16$个通道，这就限制了以全带宽与CPU连接的GPU数量。毕竟，它们还需要与其他高带宽外围设备（如存储和以太网）共享链路。与RAM访问一样，由于减少了数据包的开销，因此更适合大批量数据传输。
* **以太网**，连接计算机最常用的方式。虽然它比PCIe慢得多，但它的安装成本非常低，而且具有很强的弹性，覆盖的距离也要长得多。低级服务器的典型带宽为1GBit/s。高端设备，如云中的[C5实例](https://aws.amazon.com/ec2/instance-types/c5/））提供10到100GBit/s的带宽。与以前所有的情况一样，数据传输有很大的开销。请注意，原始以太网几乎从不被直接使用，而是在物理互连之上使用执行的协议（例如UDP或TCP/IP)。这进一步增加了开销。与PCIe类似，以太网旨在连接两个设备，例如计算机和交换机。
* **交换机**，一种连接多个设备的方式，该连接方式下的任何一对设备都可以同时执行（通常是全带宽）点对点连接。例如，以太网交换机可能以高带宽连接$40$台服务器。请注意，交换机并不是传统计算机网络所独有的。甚至PCIe通道也可以是[可交换的](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)，例如：[P2实例](https://aws.amazon.com/ec2/instance-types/p2/)就是将大量GPU连接到主机处理器。
* **NVLink**，是PCIe的替代品，适用于非常高带宽的互连。它为每条链路提供高达300Gbit/s的数据传输速率。服务器GPU（Volta V100）有六个链路。而消费级GPU（RTX 2080Ti）只有一个链路，运行速度也降低到100Gbit/s。我们建议使用[NCCL](https://github.com/NVIDIA/nccl)来实现GPU之间的高速数据传输。
  - **NCCL**是一个强大的库，可用于高效的GPU间通信，无论是在单节点还是跨多个节点的环境中。它优化了通信协议，以实现在各种硬件平台上的高带宽传输，是深度学习和其他GPU密集型应用的重要工具。
  - NCCL是一个独立的GPU标准通信例程库，实现了all-reduce、all-gather、reduce、broadcast、reduce-scatter以及任何基于发送/接收的通信模式。
  - 该模块已经过优化，可以在使用PCIe， NVLink， NVswitch的平台上实现高带宽，以及使用InfiniBand或TCP/IP套接字的网络。NCCL支持安装在单个节点或跨多个节点的任意数量的GPU，并且可以用于单进程或多进程（例如MPI）应用程序。

## 12.4.7 更多延迟信息

 表12.4.1和 表12.4.2中的小结来自[Eliot Eshelman](https://gist.github.com/eshelman)，他们将数字的更新版本保存到[GitHub gist](https://gist.github.com/eshelman/343a1c46cb3fba142c1afdcdeec17646)。

<center>表12.4.1 常见延迟</center>

| Action | Time | Notes |
| :----------------------------------------- | -----: | :---------------------------------------------- |
| L1 cache reference/hit                     | 1.5 ns | 4 cycles                                        |
| Floating-point add/mult/FMA                | 1.5 ns | 4 cycles                                        |
| L2 cache reference/hit                     |   5 ns | 12 ~ 17 cycles                                  |
| Branch mispredict                          |   6 ns | 15 ~ 20 cycles                                  |
| L3 cache hit (unshared cache)              |  16 ns | 42 cycles                                       |
| L3 cache hit (shared in another core)      |  25 ns | 65 cycles                                       |
| Mutex lock/unlock                          |  25 ns |                                                 |
| L3 cache hit (modified in another core)    |  29 ns | 75 cycles                                       |
| L3 cache hit (on a remote CPU socket)      |  40 ns | 100 ~ 300 cycles (40 ~ 116 ns)                  |
| QPI hop to a another CPU (per hop)         |  40 ns |                                                 |
| 64MB memory ref. (local CPU)          |  46 ns | TinyMemBench on Broadwell E5-2690v4             |
| 64MB memory ref. (remote CPU)         |  70 ns | TinyMemBench on Broadwell E5-2690v4             |
| 256MB memory ref. (local CPU)         |  75 ns | TinyMemBench on Broadwell E5-2690v4             |
| Intel Optane random write                  |  94 ns | UCSD Non-Volatile Systems Lab                   |
| 256MB memory ref. (remote CPU)        | 120 ns | TinyMemBench on Broadwell E5-2690v4             |
| Intel Optane random read                   | 305 ns | UCSD Non-Volatile Systems Lab                   |
| Send 4KB over 100 Gbps HPC fabric          |   1 μs | MVAPICH2 over Intel Omni-Path                   |
| Compress 1KB with Google Snappy            |   3 μs |                                                 |
| Send 4KB over 10 Gbps ethernet             |  10 μs |                                                 |
| Write 4KB randomly to NVMe SSD             |  30 μs | DC P3608 NVMe SSD (QOS 99% is 500μs)            |
| Transfer 1MB to/from NVLink GPU            |  30 μs | ~33GB/s on NVIDIA 40GB NVLink                 |
| Transfer 1MB to/from PCI-E GPU             |  80 μs | ~12GB/s on PCIe 3.0 x16 link                  |
| Read 4KB randomly from NVMe SSD            | 120 μs | DC P3608 NVMe SSD (QOS 99%)                     |
| Read 1MB sequentially from NVMe SSD        | 208 μs | ~4.8GB/s DC P3608 NVMe SSD                    |
| Write 4KB randomly to SATA SSD             | 500 μs | DC S3510 SATA SSD (QOS 99.9%)                   |
| Read 4KB randomly from SATA SSD            | 500 μs | DC S3510 SATA SSD (QOS 99.9%)                   |
| Round trip within same datacenter          | 500 μs | One-way ping is ~250μs                          |
| Read 1MB sequentially from SATA SSD        |   2 ms | ~550MB/s DC S3510 SATA SSD                    |
| Read 1MB sequentially from disk            |   5 ms | ~200MB/s server HDD                           |
| Random Disk Access (seek+rotation)         |  10 ms |                                                 |
| Send packet CA->Netherlands->CA            | 150 ms |                                                 |


<center>表12.4.2 NVIDIA Tesla GPU的延迟</center>

| Action | Time | Notes |
| :------------------------------ | -----: | :---------------------------------------- |
| GPU Shared Memory access        |  30 ns | 30~90 cycles (bank conflicts add latency) |
| GPU Global Memory access        | 200 ns | 200~800 cycles                            |
| Launch CUDA kernel on GPU       |  10 μs | Host CPU instructs GPU to start kernel    |
| Transfer 1MB to/from NVLink GPU |  30 μs | ~33GB/s on NVIDIA 40GB NVLink           |
| Transfer 1MB to/from PCI-E GPU  |  80 μs | ~12GB/s on PCI-Express x16 link         |


## 小结

* 设备有运行开销。因此，数据传输要争取量大次少而不是量少次多。这适用于RAM、固态驱动器、网络和GPU。
* 矢量化是性能的关键。确保充分了解你的加速器的特定功能。例如，一些Intel Xeon CPU特别适用于INT8操作，NVIDIA Volta GPU擅长FP16矩阵操作，NVIDIA Turing擅长FP16、INT8和INT4操作。
* 在训练过程中数据类型过小导致的数值溢出可能是个问题（在推断过程中则影响不大）。
* 数据混叠现象会导致严重的性能退化。$64$位CPU应该按照$64$位边界进行内存对齐。在GPU上建议保持卷积大小对齐，例如：与张量核对齐。
* 将算法与硬件相匹配（例如，内存占用和带宽）。将命中参数装入缓存后，可以实现很大数量级的加速比。
* 在验证实验结果之前，我们建议先在纸上勾勒出新算法的性能。关注的原因是数量级及以上的差异。
* 使用调试器跟踪调试寻找性能的瓶颈。
* 训练硬件和推断硬件在性能和价格方面有不同的优点。
